In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import load_wine
from torch.utils.tensorboard import SummaryWriter
from google.cloud import storage
import numpy as np
from io import BytesIO
import pandas as pd
import os

Download dataset from Vertex AI Datasets (with direct Google Cloud Storage link)

In [2]:
storage_client = storage.Client()
blob = storage_client.bucket("wine-quality-model").blob("wine-quality.csv")
data = pd.read_csv(BytesIO(blob.download_as_string()))

Split the data

In [3]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

Train an elastic net

In [4]:
alpha = 0.5
l1_ratio = 0.5

lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
lr.fit(train_x, train_y)

predicted_qualities = lr.predict(test_x)

Evaluate model performance

In [5]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7911824863420315
  MAE: 0.6114337698859303
  R2: 0.12989214160333407


Write to Tensorboard log. You can see the results in local Tensorboard by running `tensorboard --logdir runs/`

In [6]:
writer = SummaryWriter()

writer.add_hparams(
    {
        "alpha": alpha,
        "l1_ratio": l1_ratio
    },
    {
        "rmse": rmse,
        "mae": mae,
        "r2": r2
    }
)

writer.close()

Upload Tensorboard logs to Vertex AI

In [7]:
log_dir = writer.log_dir
run_name = os.path.basename(log_dir)

In [8]:
log_dir, run_name

('runs/May27_10-43-01_lowl-thinkpad', 'May27_10-43-01_lowl-thinkpad')

In [9]:
!tb-gcp-uploader --tensorboard_resource_name projects/fuzzylabs/locations/europe-west4/tensorboards/wine-quality-tensorboard --experiment_name="$run_name" --logdir=$log_dir --one_shot

Traceback (most recent call last):
  File "/home/dlowl/git/mlops-gcp/examples/sklearn-wine-vertex/env/lib64/python3.8/site-packages/google/api_core/grpc_helpers.py", line 67, in error_remapped_callable
    return callable_(*args, **kwargs)
  File "/home/dlowl/git/mlops-gcp/examples/sklearn-wine-vertex/env/lib64/python3.8/site-packages/grpc/_channel.py", line 923, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/home/dlowl/git/mlops-gcp/examples/sklearn-wine-vertex/env/lib64/python3.8/site-packages/grpc/_channel.py", line 826, in _end_unary_response_blocking
    raise _InactiveRpcError(state)
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Internal error encountered."
	debug_error_string = "{"created":"@1622094186.688313931","description":"Error received from peer ipv4:74.125.205.95:443","file":"src/core/lib/surface/call.cc","file_line":1062,"grpc_message":"Internal error enc